In [ ]:
import numpy as np
from scipy.io import loadmat, savemat
from numpy.core.records import fromarrays
from itertools import compress
from tqdm import tqdm, tqdm_notebook
import os, pprint
import cv2 as cv
import pandas as pd
import h5py

In [ ]:
root_dir = "/home/yuliang/data/MultiPerson_PoseTrack_v0.1/videos_align"
mat_path = "/home/yuliang/data/MultiPerson_PoseTrack_v0.1/MultiPerson_PoseTrack_v0.1.mat"
bonn_mat = loadmat(mat_path)['RELEASE']
is_train = bonn_mat['is_train'][0,0]

video_names = [video_name[0][0] for video_name in bonn_mat['annolist'][0,0]['name']]
for idx,video_name in enumerate(video_names):
    if len(video_name) == 6:
        video_names[idx] = '000'+video_name

video_frames = [video_frame[0][0] for video_frame in bonn_mat['annolist'][0,0]['num_frames']]
video_persons = [video_frame[0][0] for video_frame in bonn_mat['annolist'][0,0]['num_persons']]

train_names = list(compress(video_names[:], is_train))
test_names = [x for x in video_names if x not in train_names]

train_frames = list(compress(video_frames[:], is_train))
test_frames = list(compress(video_frames[:], 1-is_train))

train_persons = list(compress(video_persons[:], is_train))
test_persons = list(compress(video_persons[:], 1-is_train))

In [ ]:
print video_names

In [ ]:
print np.array([i[0][0] for i in bonn_mat['annolist'][0][0]['annopoints'][0][0][0][0][0][0]['point']['id'][0]])
print(bonn_mat['annolist'][0][0]['annopoints'][1][0].shape)
print 'point' in bonn_mat['annolist'][0][0]['annopoints'][9][0][3][0][0].dtype.fields.keys()
print len(bonn_mat['annolist'][0][0]['annopoints'][9][0][4][0][0])


In [ ]:
def convert_to_ascii(text):
    return np.array([ord(char) for char in text])

In [ ]:
print convert_to_ascii(data['image_id'][1])
# print list(set([0,1,2,3,4,5,8,9,10,11,12,13,14,15]))

In [ ]:
annot = {'visible':[], 'imgname':[],'part':[],'bndbox':[]}
full_ids = set([0,1,2,3,4,5,8,9,10,11,12,13,14,15])

for vid, vname in enumerate(tqdm_notebook(video_names)):
    for fid in xrange(video_frames[vid]):
        fname = '{:0>5}.jpg'.format(fid+1)
        imgname = os.path.join(vname,fname)
        img_height, img_width, _ = cv.imread(os.path.join(root_dir,imgname)).shape
#         img_height -= 2
#         img_width -= 2
        for pid in xrange(video_persons[vid]):
            if (len(bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0]) == 0) or \
               ('point' not in bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0].dtype.fields.keys()):
                continue
            xmin = np.min(bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['x'][0])[0][0]
            ymin = np.min(bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['y'][0])[0][0]
            xmax = np.max(bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['x'][0])[0][0]
            ymax = np.max(bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['y'][0])[0][0]
            if xmin>=xmax or ymin>=ymax:
                continue
#                 print 'error max min: {} {} {} {}'.format(xmin, ymin, xmax, ymax)
#                 print bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['x'][0]
#                 print bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['y'][0]
#                 continue
#             print xmin, ymin, xmax, ymax
            width, height = xmax-xmin, ymax-ymin
            ratio = 0.05
            bndbox = np.array([np.max([np.min([xmin-width*ratio,img_width]),1]), np.max([np.min([ymin-height*ratio,img_height]),1]),\
                               np.max([np.min([xmax+width*ratio,img_width]),1]), np.max([np.min([ymax+height*ratio,img_height]),1])])
            visible = np.array([i[0][0] for i in bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['is_visible'][0]])
            part_x = np.array([i[0][0] for i in bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['x'][0]])
            part_y = np.array([i[0][0] for i in bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['y'][0]])
            part_x[part_x>img_width] = img_width
            part_y[part_y>img_height] = img_height
            
#             part_x[part_x==0] = 1
#             part_y[part_y==0] = 1
            
            if np.ceil(bndbox[0])>img_width or np.ceil(bndbox[1])>img_height or np.ceil(bndbox[2])>img_width or np.ceil(bndbox[3])>img_height or (0 in np.ceil(bndbox))\
                    or np.ceil(bndbox[0])>=np.ceil(bndbox[2]) or np.ceil(bndbox[1])>=np.ceil(bndbox[3]):
                continue
#                 print 'error bndbox'
#             if np.max(part_x)>img_width or np.max(part_y)>img_height or np.min(part_x)<=0 or np.min(part_y)<=0:
#                 print 'error part'
            ids = set([i[0][0] for i in bonn_mat['annolist'][0][0]['annopoints'][vid][0][pid][fid][0][0]['point']['id'][0]])
            miss_ids = [list(full_ids).index(i) for i in (full_ids-ids)]
            for miss_id in sorted(miss_ids):
                visible = np.insert(visible, miss_id, 0)
                part_x = np.insert(part_x, miss_id, 0.0)
                part_y = np.insert(part_y, miss_id, 0.0)
                
            part = np.stack((part_x, part_y), axis=1)
#             print imgname
            annot['imgname'] += [convert_to_ascii(imgname)]
            annot['bndbox'] += [np.ceil(bndbox.reshape(1,4))]
            annot['visible'] += [visible.reshape(1,14)]
            annot['part'] += [part.reshape(14,2).astype(int)]

In [ ]:
a = np.array([1.2,3,4,5,9.23])
print np.ceil(a)

In [ ]:
for k in range(1000):
    print "".join([chr(i) for i in annot['imgname'][k]])

In [ ]:
with h5py.File('/home/yuliang/code/multi-human-pose/train/data/bonn-train/annot.h5','w') as f:
    f.attrs['name'] = 'bonn-train'
    for k in annot.keys():
        f[k] = np.array(annot[k])

In [ ]:
print(len("/home/yuliang/code/multi-human-pose/train/data/bonn-train/images/000000010/00000.jpg"))

In [ ]:
print len(annot['imgname'])

In [ ]:
with h5py.File('/home/yuliang/code/multi-human-pose/train/data/mpii-box/annot.h5','r') as f:
    id = 2
    print f['bndbox'][id]
    print f['part'][id]
    print "".join([chr(int(c)) for c in f['imgname'][id]])
    print len(f['bndbox'])

In [ ]:
with h5py.File('/home/yuliang/code/multi-human-pose/train/data/bonn-train/annot.h5','r') as f:
    id = 10
    print f['bndbox'][id]
    print f['part'][id]
    print "".join([chr(c) for c in f['imgname'][id]])

In [ ]:
with h5py.File('/home/yuliang/code/multi-human-pose/train/data/ai-cha/annot.h5','r') as f:
    print f['bndbox'][1]
    print f['part'][1]
    print len(f['bndbox'])

In [ ]:
img = cv.imread('/home/yuliang/data/MultiPerson_PoseTrack_v0.1/videos/000001/00002.jpg')
print(img.shape)